# **Rename Surfaces Based on Space Names**


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roruizf/OpenStudio-Toolkit/blob/main/[Surfaces]%20Rename-Surfaces-Based-on-Space-Names.ipynb)

---

<div style="background: rgba(248, 249, 250, 0); padding: 10px; border-left: 4px solid #ffc107; margin-bottom: 15px;">
  <p><strong>⚠️ Troubleshooting and FAQs:</strong> Refer to the bottom of the notebook for assistance.</p>
  <p><strong>💬 Feedback:</strong> If you have any feedback, comments, or suggestions about this notebook, feel free to reach out to <a href="mailto:roberto@r2f.solutions">me</a>.</p>
</div>

---


### **Table of Contents**
1. [Introduction](#introduction)
2. [User Inputs and Environment Setup](#user-inputs-and-environment-setup)
3. [Configure Project Folder and File Access](#configure-project-folder-and-file-access)
4. [Install and Import Libraries](#install-and-import-libraries)
5. [Load OpenStudio Model](#load-openstudio-model)
6. [Extract and Rename Surfaces](#extract-and-rename-surfaces)
7. [Update and Save the Model](#update-and-save-the-model)
8. [Troubleshooting and FAQs](#troubleshooting-and-faqs)
9. [Conclusion](#conclusion)

* * *


### **1. Introduction**  
This notebook is designed to rename surfaces in an OpenStudio model file (`.osm`) based on their associated space names. It ensures consistent naming conventions for surfaces, improving readability and analysis.  

By running this notebook, you will:  
- Load an OpenStudio model file.  
- Extract surface information and rename surfaces based on their space names.  
- Save the updated model with the new naming conventions.  

The notebook supports both Google Colab and local execution, with clear instructions for each environment.  

---



### **2. User Inputs and Environment Setup**
This section configures the environment based on whether the notebook is running in Google Colab or locally. Here you can define how the notebook will interact with files and where the output will be saved.

*   `google_colab`: Set to `True` if running in Google Colab, `False` if running locally. This affects file path handling.
*   `connect_gdrive`: Set to `True` to connect to Google Drive and access files from there. Set to `False` to upload files manually in Google Colab or use local paths when running locally.
*   `save_changes`: Set to `True` to save the updated OpenStudio model, `False` to not save.
*   `rewrite_osm_file`: If `save_changes` is `True`, set to `True` to overwrite the original `.osm` file with the changes, `False` to save the changes to a new file with "_updated" appended to the name.
*   `osm_file_path`: Provide the full path to your OpenStudio model file (`.osm`).
    *   **Google Colab + Google Drive:** Use the path format `/content/drive/MyDrive/your_folder/your_model.osm` after connecting Google Drive.
    *   **Google Colab + Manual Upload:** You can leave this as a placeholder; the code will handle the upload and set the path.
    *   **Local:** Use the appropriate local file path format for your operating system (e.g., `/path/to/your/model.osm` or `C:\path\to\your\model.osm`).

In [ ]:
# Set to True if running in Google Colab, False if running locally
google_colab = True
# Set to True to connect to Google Drive, False to upload a file manually
connect_gdrive = False
# Set to True to save the updated OpenStudio model, False to not save.
save_changes = True
# If save_changes is True, set to True to overwrite the original .osm file, False to save to a new file.
rewrite_osm_file = False

# Provide the full path to your OpenStudio model file (.osm).
# If connect_gdrive is True, use the path format '/content/drive/MyDrive/your_folder/your_model.osm'
# If google_colab is False, use the appropriate local file path format (e.g., '/path/to/your/model.osm' or 'C:\path/to/your/model.osm')
# If connect_gdrive is False (manual upload), the osm_file_path will be set after uploading the file. You can comment out or set to None

osm_file_path = '/content/drive/MyDrive/your_folder/your_model.osm'

### **3. Install and Import Libraries**
This section installs and imports the necessary libraries, including the `openstudio` package and custom modules. This is required to interact with OpenStudio model files and utilize the custom functions for extracting and renaming surfaces.

In [ ]:
# Install the openstudio package. Set the version you use
%pip install -q openstudio==3.7.0

# Install the openstudio_toolkit package from GitHub
!pip install -q git+https://github.com/roruizf/OpenStudio-Toolkit.git

# Import required libraries
import pandas as pd
import os
import sys
import openstudio

# Import custom modules
from openstudio_toolkit.osm_objects.spaces import *
from openstudio_toolkit.osm_objects.surfaces import *
from openstudio_toolkit.utils.osm_utils import *

### **4. Configure Project Folder and File Access**
This section configures the project folder path and handles file access (either through Google Drive or manual upload) based on the environment settings defined in the previous step. It ensures the notebook can correctly locate and work with your OpenStudio model file.

In [ ]:
# Configure project folder path based on the environment
if google_colab:
    if connect_gdrive:
        # If running in Google Colab and connecting to Google Drive:
        # Set project_folder_path to the directory containing the OSM file.
        project_folder_path = os.path.split(osm_file_path)[0]
        # Mount Google Drive to make files accessible.
        from google.colab import drive
        drive.mount('/content/drive')
    else:
        # If running in Google Colab and not connecting to Google Drive (manual upload):
        # Set project_folder_path to the default content directory.
        from google.colab import files
        project_folder_path = '/content'
else:
    # If running locally:
    # Set project_folder_path to the directory containing the OSM file.
    project_folder_path = os.path.split(osm_file_path)[0]

# Add the project folder path to the system path.
# This allows importing custom modules located in the project directory.
sys.path.append(project_folder_path)

# Handle file upload if not connecting to Google Drive.
if not connect_gdrive:
    # Prompt the user to upload the .osm file.
    uploaded_os_file = files.upload()
    # Update osm_file_path to the path of the uploaded file in the content directory.
    osm_file_path = os.path.join(project_folder_path, list(uploaded_os_file.keys())[0])

### **5. Load OpenStudio Model**
This section loads the OpenStudio model file (`.osm`) into the notebook.

In [ ]:
# Load the OpenStudio model from the specified file path
osm_model = load_osm_file_as_model(osm_file_path)
print(f"Model loaded from: {osm_file_path}")

### **6. Extract and Rename Surfaces**
This section extracts surface information from the model, renames surfaces based on their space names, and prepares the data for updates.

In [ ]:
# Extract all surface objects as a DataFrame
all_surface_objects_df = get_all_surface_objects_as_dataframe(osm_model)
all_surfaces_df = all_surface_objects_df[['Handle', 'Name', 'Surface Type', 'Construction Name', 'Space Name', 'Outside Boundary Condition', 'Outside Boundary Condition Object', 'Sun Exposure', 'Wind Exposure']]

# Rename columns for clarity
all_surfaces_df = all_surfaces_df.rename(columns={'Handle': 'Surface Handle', 'Name': 'Surface Name'})

# Sort the DataFrame by Handle column and reset indexes
all_surfaces_df = all_surfaces_df.sort_values(by='Surface Handle', ascending=True).reset_index(drop=True)
all_surfaces_df.head()

# Define new surface names based on space names and boundary conditions
all_surfaces_df.loc[all_surfaces_df['Outside Boundary Condition Object'].isnull(), 'New Surface Name'] = (
    all_surfaces_df['Space Name'] + "_" + all_surfaces_df['Surface Type'] + "_" + all_surfaces_df['Outside Boundary Condition']
)

# Create a dictionary mapping 'Outside Boundary Condition Object' to 'Space Name'
boundary_condition_object_to_space_name = all_surfaces_df.set_index('Surface Name')['Space Name'].to_dict()

# Apply the renaming code only to rows where 'Outside Boundary Condition Object' is not None
all_surfaces_df.loc[all_surfaces_df['Outside Boundary Condition Object'].notnull(), 'New Surface Name'] = (
    all_surfaces_df['Space Name'] + "_" + all_surfaces_df['Surface Type'] + "_" + all_surfaces_df['Outside Boundary Condition Object'].map(boundary_condition_object_to_space_name)
)

# Check for duplicates in 'New Surface Name'
duplicates = all_surfaces_df.duplicated(subset=['New Surface Name'], keep=False)
unique_values = all_surfaces_df[~duplicates]
duplicate_values = all_surfaces_df[duplicates]

# Initialize a counter dictionary to keep track of suffixes for each value
counter = {}

# Process unique values to add '_1' suffix
for idx, row in unique_values.iterrows():
    value = row['New Surface Name']
    new_value = f"{value}_1"
    all_surfaces_df.loc[idx, 'New Surface Name'] = new_value
    counter[value] = 1

# Process duplicate values to add incremental suffixes
for idx, row in duplicate_values.iterrows():
    value = row['New Surface Name']
    if value in counter:
        counter[value] += 1
    else:
        counter[value] = 1
    new_value = f"{value}_{counter[value]}"
    all_surfaces_df.loc[idx, 'New Surface Name'] = new_value


### **7. Update and Save the Model**
This section updates the surface names in the OpenStudio model and saves the changes.

In [ ]:
# Update surface names in the model
for index, row in all_surfaces_df.iterrows():
    surface = osm_model.getSurface(row['Surface Handle'])
    if row['New Surface Name'] != surface.get().name().get():
        print(f"Surface '{surface.get().name().get()}' renamed to '{row['New Surface Name']}'")
        surface.get().setName(row['New Surface Name'])

In [ ]:
# Save the updated model
if save_changes == True:
  if rewrite_osm_file == True:
    # Set the new file name to be the same as the original file name if overwriting
    new_file_name = os.path.split(osm_file_path)[-1]
  else:
    # Set the new file name by appending '_updated.osm' if not overwriting
    new_file_name = os.path.split(osm_file_path)[-1].split('.')[0] + '_updated.osm'

  # Construct the full path for the new file
  new_file_path = os.path.join(project_folder_path, new_file_name)
  # Save the model to the specified new file path
  save_model_as_osm_file(osm_model, osm_file_path, new_file_name)
  print(f"Model saved as {new_file_path}")

  # Download the updated osm file if not connected to Google Drive
  if connect_gdrive == False:
    files.download(os.path.join(project_folder_path, new_file_name))

### **8. Troubleshooting and FAQs**
#### *1. Installing Libraries*
- Ensure you have an active internet connection when running `%pip install`.
- If the installation fails, try restarting the runtime and running the cell again.

#### *2. Running Ubuntu Bash Commands*
- Commands like `!wget` and `!unzip` are executed in a shell environment. Ensure the commands are correctly formatted.

#### *3. Reading Files from Local Drive or Google Drive*
- For Google Drive: Use `drive.mount('/content/drive')` and specify the correct path.
- For local files: Upload the file manually using `files.upload()` or specify the local file path.

* * *

### **9. Conclusion**
This notebook automates the process of renaming surfaces in OpenStudio models based on their associated space names, ensuring consistent naming conventions. It is designed to be user-friendly and adaptable for both Google Colab and local environments.

* * *